In [1]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="Llama-3-Open-Ko-8B:latest")
llm.invoke("지구의 자전 주기는?")

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9544\2891556783.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="Llama-3-Open-Ko-8B:latest")


AIMessage(content='지구는 태양을 중심으로 공전하는 행성입니다. 지구는 태양과 약 1억 5천만 킬로미터 떨어져 있습니다. 지구는 태양을 한 바퀴 도는데 약 365일이 걸립니다.', additional_kwargs={}, response_metadata={'model': 'Llama-3-Open-Ko-8B:latest', 'created_at': '2025-01-07T06:33:40.25509Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 18105901300, 'load_duration': 16145700, 'prompt_eval_count': 60, 'prompt_eval_duration': 2057000000, 'eval_count': 60, 'eval_duration': 16030000000}, id='run-2c697ec4-c7be-4d41-b5ed-f90071b34a47-0')

In [2]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("천문학 분야의 전문가로서 행동해줘.<Qusetion>:{input}")
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='천문학 분야의 전문가로서 행동해줘.<Qusetion>:{input}'), additional_kwargs={})])

In [3]:
chain = prompt | llm

chain.invoke({"input": "지구의 자전 주기는?"})

AIMessage(content='지구는 태양계에서 가장 큰 행성입니다. 지구의 자전 주기는 약 23시간 56분 4초이며, 이는 태양과 달 사이의 거리보다 약 30만 배 더 짧습니다.', additional_kwargs={}, response_metadata={'model': 'Llama-3-Open-Ko-8B:latest', 'created_at': '2025-01-07T06:33:58.336611Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 15893501000, 'load_duration': 16186800, 'prompt_eval_count': 77, 'prompt_eval_duration': 1305000000, 'eval_count': 54, 'eval_duration': 14570000000}, id='run-f7539ddc-d331-4576-b7d2-94944f955e8c-0')

프롬프트, LLM, 문자열 출력 파서(StrOutputParser)를 연결하여 체인을 만들 수 있음

StrOutputParser는 모델의 출력을 문자열 형태로 파싱하여 최종 결과를 반환

In [4]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("천문학 분야의 전문가로서 행동해줘.<Qusetion>:{input}")
llm = ChatOllama(model="Llama-3-Open-Ko-8B:latest")
output_parset = StrOutputParser()

chain = prompt | llm | output_parset

chain.invoke({"input": "지구의 자전 주기는?"})

'지구는 태양계에서 가장 큰 행성입니다. 지구의 자전 주기는 약 23시간 56분 4초이며, 이는 태양과 달 사이의 거리보다 약 30만 배 더 짧습니다.'

## 1-2-2 멀티 체인(Multi-Chain)

- 여러 개의 체인을 연결하거나 복합적으로 작용하는 것은 멀티 체인(Multi-Chain) 구조를 통해 이루어짐

1. 순차적인 체인 연결


In [5]:
prompt1 = ChatPromptTemplate.from_template("translates {korean_word} to English.")
prompt2 = ChatPromptTemplate.from_template(
    "explain {english_word} using oxford dictionary to me in Korean."
)

llm = ChatOllama(model="llama3:latest")
chain1 = prompt1 | llm | StrOutputParser()

chain1.invoke({"korean_word":"미래"})


'The word "미래" (mi-re) translates to "future" in English.'

In [6]:
chain2 =(
    {"english_word":chain1}
    | prompt2
    |llm
    |StrOutputParser()
)

chain2.invoke({"korean_word":"미래"})

'The word "" (mirae) is a fascinating one!\n\nAccording to the Oxford Dictionary, you\'re correct that "" can be translated to English as:\n\n* Future\n* Tomorrow\n* Ahead\n* Next (as in, the next step or action)\n\nHowever, as you mentioned, the connotations of "" are more nuanced in Korean culture. It often carries optimistic and hopeful undertones, conveying a sense of promise and possibility.\n\nIn Korean, "" is often used to express aspirations and goals for one\'s life, such as "my future career" or "my future marriage". This usage implies a sense of hope and anticipation for the future.\n\nMoreover, when Koreans say "a brighter future lies ahead," it\'s not just about looking forward to what\'s coming next; it\'s about holding onto hope that things will improve. The word "" seems to carry a message of encouragement and positivity, urging people to strive for better times.\n\nIn contrast, when we use the English translation "future" in isolation, it might simply mean the time per

## 1-2-3. 체인을 실행하는 방법
"Runnable" 프로토콜
- invoke: 주어진 입력에 대해 체인을 호출하고, 결과를 반환. 이 메소드는 단일 입력에 대해 동기적으로 작동.
- batch: 입력 리스트에 대해 체인을 호출하고, 각 입력에 대한 결과를 리스트로 반환. 이 메소드는 여러 입력에 대해 동기적으로 작동하며, 효율적인 배치 처리를 가능하게 함.
- stream: 입력에 대해 체인을 호출하고, 결과의 조각들을 스트리밍. 이는 대용량 데이터 처리나 실시간 데이터 처리에 유용.
- 비동기 버전: ainvoke, abatch, astream 등의 메소드는 각각의 동기 버전에 대한 비동기 실행을 지원. 이를 통해 비동기 프로그래밍 패러다임을 사용하여 더 높은 처리 성능과 효율을 달성할 수 있음.


LangChain을 사용하여 커스텀 체인을 생성하는 과정
1. 필요한 컴포넌트를 정의하고, 각각 "Runnable" 인터페이스를 구현
2. 컴포넌트들을 조합하여 사용자 정의 체인을 생성
3. 생성된 체인을 사용하여 데이터 처리 작업을 수행. invoke, batch, stream 메소드를 사용하여 원하는 방식으로 데이터를 처리할 수 있음

In [7]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

# 컴포넌트 정의
prompt = ChatPromptTemplate.from_template("지구과학에서 {topic}에 대해 간단히 설명해주세요.")
llm = ChatOllama(model="Llama-3-Open-Ko-8B:latest")
output_parset = StrOutputParser()

chain = prompt | llm | output_parset

result = chain.invoke({"topic": "지구 자전"})
print("invoke 결과:", result)

invoke 결과: 지구는 태양 주위를 공전하면서도 자전을 합니다. 지구의 자전 속도는 초당 1,670km이며, 하루에 한 바퀴를 도는데 약 24시간이 걸립니다. 지구가 자전하는 이유는 지구 내부에서 발생하는 열 때문입니다.


In [8]:
#batch 메소드 사용
topics = ["지구 공전", "화산 활동", "대륙 이동"]
results = chain.batch([{"topic": t} for t in topics])
for topic, result in zip(topics, results):
    print(f"{topic} 설명: {result[:50]}...")  # 결과의 처음 50자만 출력

지구 공전 설명: 지구는 태양 주위를 1년 동안 한 바퀴 도는데요. 지구의 자전축은 약 23.5도 기울어져 ...
화산 활동 설명: 화산은 지각의 일부가 뜨거워져서 분출하는 현상입니다. 지각이 뜨겁게 되면, 마그마가 형성되...
대륙 이동 설명: 대륙 이동은 지각의 움직임으로 인한 것으로, 약 2억 년 전부터 시작되어 현재까지 계속되고...


In [9]:
# stream 메소드 사용
stream = chain.stream({"topic": "지진"})
print("stream 결과:")
for chunk in stream:
    print(chunk, end="", flush=True)
print()

stream 결과:
지진은 지각의 움직임으로 인한 진동을 말합니다. 지진이 발생하면 지표면이 흔들리고 건물이나 도로가 파괴될 수 있습니다. 지진은 크게 자연지진과 인공지진으로 나뉩니다.


In [10]:
import nest_asyncio
import asyncio

# nest_asyncio 적용 (구글 코랩 등 주피터 노트북에서 실행 필요)
nest_asyncio.apply()

# 비동기 메소드 사용 (async/await 구문 필요)
async def run_async():
    result = await chain.ainvoke({"topic": "해류"})
    print("ainvoke 결과:", result[:50], "...")

asyncio.run(run_async())


ainvoke 결과: 해류는 바다의 물이 바람이나 달의 인력 등으로 생기는 힘을 받아서 이동하는 흐름입니다. 해 ...


# 1-3. 프롬프트(Prompt)
프롬프트는 사용자와 언어 모델 간의 대화에서 질문이나 요청의 형태로 제시되는 입력문

## 1-3-1. 프롬프트 작성 원칙
1. 명확성과 구체성
2. 배경 정보를 포함
3. 간결함
4. 열린 질문 사용
5. 명확한 목표 설정
6. 언어와 문체

## 1-3-2. 프롬프트 템플릿(PromptTemplate)
- PromptTemplate은 단일 문장 또는 간단한 명령을 입력하여 단일 문장 또는 간단한 응답을 생성하는 데 사용되는 프롬프트를 구성할 수 있는 문자열 템플릿
1. 구성요소
지시 : 언어 모델에게 어떤 작업을 수행하도록 요청하는 구체적인 지시.   
예시 : 요청된 작업을 수행하는 방법에 대한 하나 이상의 예시.   
맥락 : 특정 작업을 수행하기 위한 추가적인 맥락   
질문 : 어떤 답변을 요구하는 구체적인 질문.   

2. 문자열 탬플릿
langchain_core.prompts 모듈의 PromptTemplate 클래스를 사용

In [11]:
from langchain_core.prompts import PromptTemplate

template_text = "안녕하세요, 제 이름은 {name}입니다. {age}살입니다."
prompt_template = PromptTemplate.from_template(template_text)
filled_prompt = prompt_template.format(name="홍길동", age=30)
filled_prompt

'안녕하세요, 제 이름은 홍길동입니다. 30살입니다.'

3. 프롬프트 템플릿 간의 결합
- PromptTemplate 클래스는 문자열을 기반으로 프롬프트 템플릿을 생성하고, + 연산자를 사용하여 직접 결합하는 동작을 지원   
    - 인스턴스 간의 직접적인 결합뿐만 아니라, 이들 인스턴스와 문자열로 이루어진 템플릿을 결합하여 새로운 PromptTemplate 인스턴스를 생성하는 것도 가능   

ex) 
문자열 + 문자열   
PromptTemplate + PromptTemplate   
PromptTemplate + 문자열   

In [12]:
# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n아버지를 아버지라 부를 수 없습니다.")
              + "\n\n{language}로 번역해주세요."
)

combined_prompt


PromptTemplate(input_variables=['age', 'language', 'name'], input_types={}, partial_variables={}, template='안녕하세요, 제 이름은 {name}입니다. {age}살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n{language}로 번역해주세요.')

In [13]:
combined_prompt.format(name="홍길동", age=30, language="영어")

'안녕하세요, 제 이름은 홍길동입니다. 30살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n영어로 번역해주세요.'

ChatOpenAI 인스턴스를 생성하여 프롬프트 텍스트를 전달하고, 모델의 출력을 StrOutputParser를 통해 문자열로 변환하는 LLM 체인을 구성   
invoke 메소드를 사용하여 파이프라인을 실행하고, 최종적으로 문자열 출력을 얻음   
모델의 응답은 프롬프트에 주어진 문장을 영어로 번역한 텍스트가 출력

In [14]:
from langchain_core.output_parsers import StrOutputParser

chain = combined_prompt | llm | StrOutputParser()
chain.invoke({"age":30, "language":"영어", "name":"홍길동"})

"Hello, my name is Hong Gil-dong. I am 30 years old.\nI can't call my father a father.\nPlease translate it into English."

# 1-3-3. 챗 프롬프트 템플릿 (ChatPromptTemplate)
- 대화형 상황에서 여러 메시지 입력을 기반으로 단일 메시지 응답을 생성하는 데 사용
- 대화형 모델이나 챗봇 개발에 주로 사용
- 입력은 여러 메시지를 원소로 갖는 리스트로 구성되며, 각 메시지는 역할(role)과 내용(content)으로 구성   

1. Message 유형
SystemMessage: 시스템의 기능을 설명   
HumanMessage: 사용자의 질문을 나타냄   
AIMessage: AI 모델의 응답을 제공   
FunctionMessage: 특정 함수 호출의 결과를 나타냄   
ToolMessage: 도구 호출의 결과를 나타냄   

2. 2-튜플 형태의 메시지 리스트
ChatPromptTemplate.from_messages 메서드 :  2-튜플 형태의 메시지 리스트를 입력 받아, 각 메시지의 역할(type)과 내용(content)을 기반으로 프롬프트를 구성

In [15]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 답변할 수 있습니다."),
    ("user", "{user_input}"),
])

messages = chat_prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")
messages

[SystemMessage(content='이 시스템은 천문학 질문에 답변할 수 있습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='태양계에서 가장 큰 행성은 무엇인가요?', additional_kwargs={}, response_metadata={})]

chat_prompt, llm, StrOutputParser()를 순차적인 파이프라인으로 연결하여 구성된 chain을 사용   
invoke 메소드를 호출하면 사용자 입력을 받아 언어 모델에 전달하고, 모델의 응답을 처리하여 최종 문자열 결과를 반환하는 과정을 자동화하여 수행

In [16]:
from langchain_core.output_parsers import StrOutputParser

chain = chat_prompt | llm | StrOutputParser()

chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

'지구입니다. 지구는 태양계에서 네 번째로 크고, 목성보다 약 10배 더 무겁습니다.'

3. MessagePromptTemplate 사용
ChatPromptTemplate.from_messages 메소드를 통해 시스템 메시지와 사용자 메시지 템플릿을 포함하는 챗 프롬프트를 구성 -> 이후, chat_prompt.format_messages 메서드를 사용하여 사용자의 질문을 포함한 메시지 리스트를 동적으로 생성

In [17]:
# MessagePromptTemplate 활용

from langchain_core.prompts import SystemMessagePromptTemplate,  HumanMessagePromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("이 시스템은 천문학 질문에 답변할 수 있습니다."),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

messages = chat_prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")
messages


[SystemMessage(content='이 시스템은 천문학 질문에 답변할 수 있습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='태양계에서 가장 큰 행성은 무엇인가요?', additional_kwargs={}, response_metadata={})]

In [18]:
chain = chat_prompt | llm | StrOutputParser()

chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

'지구입니다. 지구는 태양계에서 네 번째로 크고, 목성보다 약 10배 더 무겁습니다.'

# 1-3-4. Few-shot-Prompt
- Few-shot 학습은 언어 모델에 몇 가지 예시를 제공하여 특정 작업을 수행하도록 유도하는 기법

1. Few-shot 예제 포맷터 생성
- PromptTemplate 은 질문과 답변을 포함하는 간단한 구조

In [19]:
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate.from_template("질문: {question}\n{answer}")


In [20]:
examples = [
    {
        "question": "지구의 대기 중 가장 많은 비율을 차지하는 기체는 무엇인가요?",
        "answer": "지구 대기의 약 78%를 차지하는 질소입니다."
    },
    {
        "question": "광합성에 필요한 주요 요소들은 무엇인가요?",
        "answer": "광합성에 필요한 주요 요소는 빛, 이산화탄소, 물입니다."
    },
    {
        "question": "피타고라스 정리를 설명해주세요.",
        "answer": "피타고라스 정리는 직각삼각형에서 빗변의 제곱이 다른 두 변의 제곱의 합과 같다는 것입니다."
    },
    {
        "question": "지구의 자전 주기는 얼마인가요?",
        "answer": "지구의 자전 주기는 약 24시간(정확히는 23시간 56분 4초)입니다."
    },
    {
        "question": "DNA의 기본 구조를 간단히 설명해주세요.",
        "answer": "DNA는 두 개의 폴리뉴클레오티드 사슬이 이중 나선 구조를 이루고 있습니다."
    },
    {
        "question": "원주율(π)의 정의는 무엇인가요?",
        "answer": "원주율(π)은 원의 지름에 대한 원의 둘레의 비율입니다."
    }
]


3. FewShotPromptTemplate 생성
- FewShotPromptTemplate 은 예제들을 결합하고 새로운 입력을 추가하여 최종 프롬프트를 생성

In [21]:
from langchain_core.prompts import FewShotPromptTemplate

# FewShotPromptTemplate을 생성합니다.
prompt = FewShotPromptTemplate(
    examples=examples,              # 사용할 예제들
    example_prompt=example_prompt,  # 예제 포맷팅에 사용할 템플릿
    suffix="질문: {input}",          # 예제 뒤에 추가될 접미사
    input_variables=["input"],      # 입력 변수 지정
)

# 새로운 질문에 대한 프롬프트를 생성하고 출력합니다.
print(prompt.invoke({"input": "화성의 표면이 붉은 이유는 무엇인가요?"}).to_string())


질문: 지구의 대기 중 가장 많은 비율을 차지하는 기체는 무엇인가요?
지구 대기의 약 78%를 차지하는 질소입니다.

질문: 광합성에 필요한 주요 요소들은 무엇인가요?
광합성에 필요한 주요 요소는 빛, 이산화탄소, 물입니다.

질문: 피타고라스 정리를 설명해주세요.
피타고라스 정리는 직각삼각형에서 빗변의 제곱이 다른 두 변의 제곱의 합과 같다는 것입니다.

질문: 지구의 자전 주기는 얼마인가요?
지구의 자전 주기는 약 24시간(정확히는 23시간 56분 4초)입니다.

질문: DNA의 기본 구조를 간단히 설명해주세요.
DNA는 두 개의 폴리뉴클레오티드 사슬이 이중 나선 구조를 이루고 있습니다.

질문: 원주율(π)의 정의는 무엇인가요?
원주율(π)은 원의 지름에 대한 원의 둘레의 비율입니다.

질문: 화성의 표면이 붉은 이유는 무엇인가요?


4. 예제 선택기 사용하기
-  분석: SemanticSimilarityExampleSelector는 입력 질문과 가장 유사한 예제를 선택

In [22]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings

# HuggingFace 임베딩 모델 초기화
embeddings = HuggingFaceEmbeddings(
    model_name='BAAI/bge-m3',
    model_kwargs={'device': 'cuda'},  # GPU 사용
    encode_kwargs={'normalize_embeddings': True}  # 임베딩 정규화
)

# SemanticSimilarityExampleSelector를 초기화
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,  # 사용할 예제들
    embeddings,  # HuggingFace 임베딩 모델 사용
    Chroma,  # 벡터 저장소
    k=1  # 선택할 예제 수
)

# 새로운 질문에 대해 가장 유사한 예제를 선택
question = "화성의 표면이 붉은 이유는 무엇인가요?"
selected_examples = example_selector.select_examples({"question": question})

print(f"입력과 가장 유사한 예제: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9544\3304479034.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


입력과 가장 유사한 예제: 화성의 표면이 붉은 이유는 무엇인가요?


answer: 광합성에 필요한 주요 요소는 빛, 이산화탄소, 물입니다.
question: 광합성에 필요한 주요 요소들은 무엇인가요?


채팅 모델에서 Few-shot 예제 사용하기
1. 고정 예제 사용하기

In [23]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# 예제 정의
examples = [
    {"input": "지구의 대기 중 가장 많은 비율을 차지하는 기체는 무엇인가요?", "output": "질소입니다."},
    {"input": "광합성에 필요한 주요 요소들은 무엇인가요?", "output": "빛, 이산화탄소, 물입니다."},
]

# 예제 프롬프트 템플릿 정의
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# Few-shot 프롬프트 템플릿 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 템플릿 생성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 과학과 수학에 대해 잘 아는 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)


from langchain_community.chat_models import ChatOllama

model = ChatOllama(model="Llama-3-Open-Ko-8B:latest")
chain = final_prompt | model

result = chain.invoke({"input": "지구의 자전 주기는 얼마인가요?"})
print(result.content)

24시간입니다.


2. 동적 Few-shot 프롬프팅
- 예제 선택기(ExampleSelector)를 사용해서 입력에 따라 전체 예제 세트에서 가장 관련성 높은 예제만 선택하여 보여주는 방법

In [25]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
# 더 많은 예제 추가
examples = [
    {"input": "지구의 대기 중 가장 많은 비율을 차지하는 기체는 무엇인가요?", "output": "질소입니다."},
    {"input": "광합성에 필요한 주요 요소들은 무엇인가요?", "output": "빛, 이산화탄소, 물입니다."},
    {"input": "피타고라스 정리를 설명해주세요.", "output": "직각삼각형에서 빗변의 제곱은 다른 두 변의 제곱의 합과 같습니다."},
    {"input": "DNA의 기본 구조를 간단히 설명해주세요.", "output": "DNA는 이중 나선 구조를 가진 핵산입니다."},
    {"input": "원주율(π)의 정의는 무엇인가요?", "output": "원의 둘레와 지름의 비율입니다."},
]

# 벡터 저장소 생성
to_vectorize = [" ".join(example.values()) for example in examples]

# HuggingFace 임베딩 모델 초기화
embeddings = HuggingFaceEmbeddings(
    model_name='BAAI/bge-m3',
    model_kwargs={'device': 'cuda'},  # GPU 사용
    encode_kwargs={'normalize_embeddings': True}  # 임베딩 정규화
)

vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

# 예제 선택기 생성
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# Few-shot 프롬프트 템플릿 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

# 최종 프롬프트 템플릿 생성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 과학과 수학에 대해 잘 아는 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델과 체인 생성
chain = final_prompt | ChatOllama(model="Llama-3-Open-Ko-8B:latest")

result = chain.invoke("태양계에서 가장 큰 행성은 무엇인가요?")
print(result.content)


KeyError: 'input'

In [27]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.chat_models import ChatOllama
import torch
# 예제 데이터 추가
examples = [
    {"input": "지구의 대기 중 가장 많은 비율을 차지하는 기체는 무엇인가요?", "output": "질소입니다."},
    {"input": "광합성에 필요한 주요 요소들은 무엇인가요?", "output": "빛, 이산화탄소, 물입니다."},
    {"input": "피타고라스 정리를 설명해주세요.", "output": "직각삼각형에서 빗변의 제곱은 다른 두 변의 제곱의 합과 같습니다."},
    {"input": "DNA의 기본 구조를 간단히 설명해주세요.", "output": "DNA는 이중 나선 구조를 가진 핵산입니다."},
    {"input": "원주율(π)의 정의는 무엇인가요?", "output": "원의 둘레와 지름의 비율입니다."},
]

# 벡터화할 텍스트 생성
to_vectorize = [" ".join(example.values()) for example in examples]

# Hugging Face 임베딩 모델 초기화
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",  # 모델 이름
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},  # GPU/CPU 자동 선택
    encode_kwargs={"normalize_embeddings": True}  # 정규화 활성화
)

# Chroma 벡터스토어 생성
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

# 예제 선택기 생성
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2  # 가장 유사한 2개의 예제 선택
)

# Few-shot 프롬프트 템플릿 생성
example_prompt = ChatPromptTemplate.from_messages(
    [("human", "{input}"), ("ai", "{output}")]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt
)

# 최종 프롬프트 템플릿 생성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 과학과 수학에 대해 잘 아는 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}")
    ]
)

# 모델과 체인 생성
chain = final_prompt | ChatOllama(model="Llama-3-Open-Ko-8B:latest")

# 체인 실행
result = chain.invoke("태양계에서 가장 큰 행성은 무엇인가요?")
print(result.content)


KeyError: 'input'

In [28]:
from langchain_core.prompts import PromptTemplate

# 기본 프롬프트 템플릿 정의
prompt = PromptTemplate.from_template("지구의 {layer}에서 가장 흔한 원소는 {element}입니다.")

# 'layer' 변수에 '지각' 값을 미리 지정하여 부분 포맷팅
partial_prompt = prompt.partial(layer="지각")

# 나머지 'element' 변수만 입력하여 완전한 문장 생성
print(partial_prompt.format(element="산소"))


지구의 지각에서 가장 흔한 원소는 산소입니다.


In [29]:
# 프롬프트 초기화 시 부분 변수 지정
prompt = PromptTemplate(
    template="지구의 {layer}에서 가장 흔한 원소는 {element}입니다.",
    input_variables=["element"],  # 사용자 입력이 필요한 변수
    partial_variables={"layer": "맨틀"}  # 미리 지정된 부분 변수
)

# 남은 'element' 변수만 입력하여 문장 생성
print(prompt.format(element="규소"))


지구의 맨틀에서 가장 흔한 원소는 규소입니다.


In [30]:
from datetime import datetime

# 현재 계절을 반환하는 함수 정의
def get_current_season():
    month = datetime.now().month
    if 3 <= month <= 5:
        return "봄"
    elif 6 <= month <= 8:
        return "여름"
    elif 9 <= month <= 11:
        return "가을"
    else:
        return "겨울"

# 함수를 사용한 부분 변수가 있는 프롬프트 템플릿 정의
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}입니다.",
    input_variables=["phenomenon"],  # 사용자 입력이 필요한 변수
    partial_variables={"season": get_current_season}  # 함수를 통해 동적으로 값을 생성하는 부분 변수
)

# 'phenomenon' 변수만 입력하여 현재 계절에 맞는 문장 생성
print(prompt.format(phenomenon="꽃가루 증가"))


겨울에 일어나는 대표적인 지구과학 현상은 꽃가루 증가입니다.
